In [8]:
# custom imports
from regression_class import RedditRegression as RR

# data saving imports
import pickle
import os

# time tracking
from datetime import datetime as dt

import pandas as pd

In [2]:
DOMAINS_INFILE = "../top10milliondomains.csv/top10milliondomains.csv"
domain_rank = pd.read_csv(DOMAINS_INFILE)

In [2]:
# PARAMS_DICT_INFILE = f"{OUTDIR}/input_params.p"
REGRESSION_INFILE = "regression_thread_data.p"
THREAD_INFILE = "clean_5_thread_data.p"

In [3]:
regdfs = pickle.load(open(REGRESSION_INFILE, 'rb'))
threaddfs = pickle.load(open(THREAD_INFILE, 'rb'))

In [4]:
domain_rank = domain_rank[['Domain', 'Open Page Rank']].rename(columns = {
    'Domain': 'domain',
    'Open Page Rank': 'open_page_rank'
})


In [7]:
domain_rank.open_page_rank.unique()

array([10.  ,  9.88,  9.83,  9.65,  9.64,  9.57,  9.24,  9.16,  9.15,
        9.14,  9.09,  9.02,  8.99,  8.96,  8.93,  8.9 ,  8.89,  8.85,
        8.76,  8.74,  8.71,  8.58,  8.54,  8.48,  8.47,  8.45,  8.41,
        8.38,  8.36,  8.35,  8.33,  8.32,  8.29,  8.27,  8.25,  8.23,
        8.22,  8.21,  8.2 ,  8.16,  8.15,  8.11,  8.1 ,  8.09,  8.07,
        8.05,  8.04,  8.03,  8.  ,  7.98,  7.97,  7.94,  7.93,  7.91,
        7.88,  7.87,  7.84,  7.83,  7.82,  7.78,  7.76,  7.75,  7.74,
        7.72,  7.71,  7.7 ,  7.69,  7.67,  7.66,  7.65,  7.63,  7.62,
        7.6 ,  7.59,  7.57,  7.56,  7.55,  7.52,  7.51,  7.49,  7.48,
        7.47,  7.46,  7.44,  7.43,  7.42,  7.41,  7.4 ,  7.39,  7.38,
        7.37,  7.36,  7.35,  7.34,  7.33,  7.32,  7.31,  7.3 ,  7.28,
        7.27,  7.26,  7.25,  7.24,  7.23,  7.22,  7.21,  7.2 ,  7.18,
        7.17,  7.16,  7.15,  7.13,  7.12,  7.11,  7.1 ,  7.09,  7.08,
        7.07,  7.06,  7.05,  7.04,  7.03,  7.02,  7.01,  7.  ,  6.99,
        6.98,  6.97,

In [85]:
domain_pagerank_dict = dict(zip(domain_rank.domain,domain_rank.open_page_rank))

In [19]:
def change_reddit_domains(domains_col):
    to_alter = domains_col[domains_col.str.contains('self.')]
    to_alter = to_alter.str.removeprefix('self.') + '.reddit.com'
    domains_col[domains_col.str.contains('self.')] = to_alter
    return domains_col

def get_pagerank_scores(domain_col):
    domain_col = change_reddit_domains(domain_col)
    new_domain_col = domain_col.map(domain_pagerank_dict, na_action=None).fillna(0)
    return new_domain_col


In [21]:

for key in threaddfs:
    threaddfs[key]['domain'] = change_reddit_domains(threaddfs[key].domain)

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [15]:
new_reg_dfs = {}
for key in threaddfs:
    posts = threaddfs[key][threaddfs[key].thread_id == threaddfs[key].id]
    posts['domain_pagerank'] = get_pagerank_scores(posts.domain)
    new_reg_dfs[key] = pd.merge(regdfs[key], posts[['thread_id', 'domain', 'domain_pagerank']], how='left', on='thread_id')

C:\Users\snuzz\AppData\Local\Temp\ipykernel_17828\2515556108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domains_col[domains_col.str.contains('self.')] = to_alter
C:\Users\snuzz\AppData\Local\Temp\ipykernel_17828\2515556108.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domains_col[domains_col.str.contains('self.')] = to_alter


NameError: name 'domain_pagerank_dict' is not defined

In [89]:
pickle.dump(new_reg_dfs, open('regression_thread_data_april_2024.p', 'wb'))

In [104]:
post_domains = posts[['thread_id', 'domain']].groupby('domain').count().rename(columns={'thread_id': 'domain_count'})

In [105]:
post_domains.sort_values(by='domain_count', ascending=False).head(30)

,domain_count
domain,
thehill.com,3317
cnn.com,2785
nytimes.com,2273
washingtonpost.com,2224
youtube.com,2007
youtu.be,1660
apnews.com,1620
reuters.com,1581
independent.co.uk,1555


In [108]:
post_domains.domain_count.sum()

65343

In [109]:
post_domains.domain_count.median()

1.0

In [110]:
post_domains.domain_count.mean()

15.964573662350354

In [113]:
post_domains.domain_count.quantile(q=0.90)

11.0

In [120]:
((post_domains.domain_count/post_domains.domain_count.sum())*100).sort_values(ascending=False).head(30)

domain
thehill.com            5.076290
cnn.com                4.262124
nytimes.com            3.478567
washingtonpost.com     3.403578
youtube.com            3.071484
youtu.be               2.540440
apnews.com             2.479225
reuters.com            2.419540
independent.co.uk      2.379750
politico.com           2.096629
newsweek.com           2.017048
theguardian.com        1.972667
nbcnews.com            1.813507
thedailybeast.com      1.660469
businessinsider.com    1.645165
twitter.com            1.634452
google.com             1.380408
foxnews.com            1.244204
cnbc.com               1.212066
usatoday.com           1.158502
huffpost.com           1.137077
npr.org                1.033010
axios.com              1.016176
edition.cnn.com        0.985568
commondreams.org       0.829469
cbsnews.com            0.824878
nypost.com             0.823348
bloomberg.com          0.780497
abcnews.go.com         0.725403
salon.com              0.676431
Name: domain_count, dtype: float6

In [9]:
threaddfs = pickle.load(open('regression_thread_data_april_2024.p', 'rb'))

In [10]:
threaddfs['crypto']

,thread_id,thread_size,authors,timestamp,author,score,subject_sentiment_score,sentiment_sign,sentiment_magnitude,success,domain,domain_pagerank
0,xsglev,2541,439,2022-10-01 00:00:13,8ac426fd80f0ea3761bdcd7f32591b09ce1c1366e59ef15c,1,0.0000,0.0,0.0000,1,CryptoCurrency.reddit.com,0.00
1,xsgqgi,1,1,2022-10-01 00:06:27,41745e3272e1a87209a6a5749a2dafbd5cd3cf893eef1f2b,1,0.7096,1.0,0.7096,0,CryptoCurrency.reddit.com,0.00
2,xsgqkx,1,1,2022-10-01 00:06:36,083ef865b12614c169cdba4e71069fd8be0cc39865222e24,1,0.3804,1.0,0.3804,0,lyksoomu.com,3.16
3,xsgsb3,1,1,2022-10-01 00:08:49,d003c719ed03f48fc76d427eb3056f79590f84c7351b32a8,1,0.0000,0.0,0.0000,0,CryptoCurrency.reddit.com,0.00
4,xsgz7t,22,16,2022-10-01 00:17:36,1230e32c76caf095a665b716ae1b0101adc0161facd5ea70,1,-0.6858,-1.0,0.6858,1,fool.com,5.94
...,...,...,...,...,...,...,...,...,...,...,...,...
14813,yhtvpj,1,1,2022-10-30 23:31:16,8e7e8bbde1a15e3765665fa16984443e5b002c3d9b647da4,1,0.0000,0.0,0.0000,0,CryptoCurrency.reddit.com,0.00
14814,yhu40u,1,1,2022-10-30 23:39:29,81f017b4a3f74094577db57f3e8715d10a728b2b514df828,1,0.0000,0.0,0.0000,0,youtube.com,10.00
14815,yhu53f,1,1,2022-10-30 23:40:33,b241d09cd6f38fc1c7759070a5c4c511e1a41cef0ff9a390,1,0.2263,1.0,0.2263,0,CryptoCurrency.reddit.com,0.00
14816,yhu621,1,1,2022-10-30 23:41:29,4b5c89acc8cde4c21ae333b70197c9b5e7557afb2a27100f,1,0.4404,1.0,0.4404,0,CryptoCurrency.reddit.com,0.00


In [31]:
true_false_map = {
    True: 0,
    False: 1
}

name_map = {
    'politics': 'politics',
    'crypto': 'CryptoCurrency',
    'conspiracy': 'conspiracy',
    'books': 'books'
}

In [32]:
threaddfs['books'].domain

0                  deadline.com
1         thebookreviewcrew.com
2            story-arc-blog.com
3              books.reddit.com
4              books.reddit.com
                 ...           
4771           books.reddit.com
4772    saturdayeveningpost.com
4773           books.reddit.com
4774           books.reddit.com
4775           books.reddit.com
Name: domain, Length: 4776, dtype: object

In [17]:
(threaddfs['crypto'].domain == "CryptoCurrency.reddit.com").map(true_false_map)

0        0
1        0
2        1
3        0
4        1
        ..
14813    0
14814    1
14815    0
14816    0
14817    0
Name: domain, Length: 14818, dtype: int64

In [35]:
for subreddit in threaddfs:
    if subreddit != "thedonald":
        threaddfs[subreddit]['external_domain'] = (threaddfs[subreddit].domain == f"{name_map[subreddit]}.reddit.com").map(true_false_map)

In [40]:
threaddfs.pop('thedonald')
threaddfs.pop('books')
threaddfs.keys()

dict_keys(['conspiracy', 'crypto', 'politics'])

In [41]:
pickle.dump(threaddfs, open('regression_thread_data_dec_24_ext_domains.p', 'wb'))